conda install pymysq

# Python MySQL

In [2]:
import json
import pymysql

In [3]:
with open('mysql.json','r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [4]:
conn = pymysql.connect(
host = config['host'],
user = config['user'],
password = config['password'],
database = config['database'],
port = config['port'],
)

## 테이블 생성
sql로 db에 테이블 생성

In [7]:
sql_user = """
CREATE TABLE if NOT EXISTS users(
	uid VARCHAR(20) NOT NULL PRIMARY KEY,
	pwd CHAR(44),
	uname VARCHAR(20) NOT NULL,
	reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
	is_deleted INT DEFAULT 0
);
"""

In [8]:
cur = conn.cursor()
cur.execute(sql_user)

0

## 테이블 변경
cur.execute() 괄호안 sql 이용

In [9]:
# email column 추가하여 테이블 변경
cur.execute('alter table users add column email varchar(40);') 

0

### 값 추가

In [11]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin','관리자');")

1

In [14]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows: 
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 49, 56), 0, None)


In [15]:
# 입력한 데이터를 즉시 flush하려면 conn.commit()을 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname)
    VALUES('eskim','김은숙'),('wjlee','이우정'); """
cur.execute(sql_insert_multi)
conn.commit()

In [17]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows: 
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 49, 56), 0, None)
('eskim', None, '김은숙', datetime.datetime(2021, 6, 9, 10, 56, 51), 0, None)
('wjlee', None, '이우정', datetime.datetime(2021, 6, 9, 10, 56, 51), 0, None)


In [18]:
# is_deleted : 삭제된것은 1 삭제 안된것은 0

sql_select = """
    SELECT uid,uname,DATE_FORMAT(reg_date,"%Y-%m-%d %h:%i")AS reg_date, is_deleted, email FROM users; 
    """
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows: 
    print(row)

('admin', '관리자', '2021-06-09 10:49', 0, None)
('eskim', '김은숙', '2021-06-09 10:56', 0, None)
('wjlee', '이우정', '2021-06-09 10:56', 0, None)


In [21]:
# 파라메터 이용, placeholder

uid = 'diy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s,%s);"
cur.execute(sql, (uid,uname))
conn.commit()

In [33]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s,'홍길동');"
cur.execute(sql, (uid,) ) # 튜플로 넣어주어야함
conn.commit()

In [29]:
users =(('jbpark','박재범'), ('vicenzo','빈센조')) 
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s,%s);"
for user in users:
    cur.execute(sql, user)
conn.commit()

## 데이터 삭제

In [34]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin' ;"
cur.execute(sql) 
conn.commit()

### 데이터프레임에서 데이터 가져와 db에 저장 

### - 데이터프레임 생성

In [35]:
import pandas as pd
uids =['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
unames= ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids, 'uname':unames})  # 데이터프레임만듬 
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


### -데이터프레임에서 가져와 db에 적재

In [36]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s,%s);"
for i in df.index:
    uid = df.uid[i]
    uname = df.uname[i]
    cur.execute(sql, (uid, uname))
conn.commit()

## 데이터 수정 

In [40]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "update users set email = %s where uid = %s;"
cur.execute(sql, (email, uid))
conn.commit()

# quiz]

In [41]:
import pandas as pd
uids =['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
emails= ['djy@email.com','eskim@email.com','gdhong@email.com','jbpark@email.com','vincenzo@email.com','wjlee@email.com']
df = pd.DataFrame({'uid':uids, 'email':emails})  # 데이터프레임만듬 
df

,uid,email
0,djy,djy@email.com
1,eskim,eskim@email.com
2,gdhong,gdhong@email.com
3,jbpark,jbpark@email.com
4,vincenzo,vincenzo@email.com
5,wjlee,wjlee@email.com


In [46]:
cur = conn.cursor()
sql = "update users set email = %s where uid = %s;"
for i in df.index:
    cur.execute(sql, (df.email[i], df.uid[i]))
conn.commit()

In [47]:
# sol) case2 ) df만들지 않고 update
cur = conn.cursor()
sql = "update users set email = %s where uid = %s;"
for i in df.index:
    uid = df.uid[i]
    email = f'{uid}@email.com'
    cur.execute(sql, (email, uid))
conn.commit()

## 데이터 조회

In [56]:
sql ="""
    SELECT uid,uname, email,
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i')AS reg_date
    FROM users WHERE is_deleted=0; 
    """

In [57]:
# 한 레코드 조회 : cur.fetchone()

cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone() 
result

('admin', '관리자', 'admin@korea.com', '2021-06-09 10:49')

In [58]:
# n개의 레코드 조회
cur.fetchmany(2) # 앞에서 읽은 데이터 다음데이터 부터 나옴. 즉 관리자 읽었으므로 그다음 적재된데이터 호출

(('djy', '대조영', 'djy@email.com', '2021-06-09 11:30'),
 ('eskim', '김은숙', 'eskim@email.com', '2021-06-09 11:30'))

In [59]:
cur.fetchmany(1) # 관리자, 대조영, 김은숙 다음 데이터인 홍길동 1개를 호출

(('gdhong', '홍길동', 'gdhong@email.com', '2021-06-09 11:30'),)

In [60]:
#모두 조회
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall() 
for row in results:
    print(row)

('admin', '관리자', 'admin@korea.com', '2021-06-09 10:49')
('djy', '대조영', 'djy@email.com', '2021-06-09 11:30')
('eskim', '김은숙', 'eskim@email.com', '2021-06-09 11:30')
('gdhong', '홍길동', 'gdhong@email.com', '2021-06-09 11:30')
('jbpark', '박재범', 'jbpark@email.com', '2021-06-09 11:30')
('vincenzo', '빈센조', 'vincenzo@email.com', '2021-06-09 11:30')
('wjlee', '이우정', 'wjlee@email.com', '2021-06-09 11:30')


## QUI] 조회한 정보를 데이터 프레임으로 만들기

In [63]:
## 
uid, uname, email, reg_date=[],[],[],[]
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall() 
for row in results:
    for i in range(4):
     if i == 0: uid.append(row[i])
     if i == 1: uname.append(row[i])
     if i == 2: email.append(row[i])
     if i == 3: reg_date.append(row[i])

In [65]:
ex = pd.DataFrame({
    'id' : uid,
    'name' : uname,
    'email' : email,
    'reg_date' : reg_date
})
ex

,id,name,email,reg_date
0,admin,관리자,admin@korea.com,2021-06-09 10:49
1,djy,대조영,djy@email.com,2021-06-09 11:30
2,eskim,김은숙,eskim@email.com,2021-06-09 11:30
3,gdhong,홍길동,gdhong@email.com,2021-06-09 11:30
4,jbpark,박재범,jbpark@email.com,2021-06-09 11:30
5,vincenzo,빈센조,vincenzo@email.com,2021-06-09 11:30
6,wjlee,이우정,wjlee@email.com,2021-06-09 11:30


In [67]:
# SOL) 
uid, uname, email, reg_date=[],[],[],[]
for row in results:
    uid.append(row[0])
    uname.append(row[1])
    email.append(row[2])
    reg_date.append(row[3])

ex = pd.DataFrame({
    'uid' : uid,
    'name' : uname,
    'email' : email,
    'reg_date' : reg_date
})
ex

,uid,name,email,reg_date
0,admin,관리자,admin@korea.com,2021-06-09 10:49
1,djy,대조영,djy@email.com,2021-06-09 11:30
2,eskim,김은숙,eskim@email.com,2021-06-09 11:30
3,gdhong,홍길동,gdhong@email.com,2021-06-09 11:30
4,jbpark,박재범,jbpark@email.com,2021-06-09 11:30
5,vincenzo,빈센조,vincenzo@email.com,2021-06-09 11:30
6,wjlee,이우정,wjlee@email.com,2021-06-09 11:30


## 데이터 검색 

In [70]:
keyword = '김'
sql ="""
    SELECT uid
    FROM users
    WHERE is_deleted=0 and uname like %s; 
    """

In [73]:
cur = conn.cursor()
cur.execute(sql,keyword+'%') # wild character, 즉 %도 붙여주어야함
result = cur.fetchone() #결과가 하나라도 튜플로 데이터를 반환
result # tuple 이므로 원하는 하나의 데이터를 뽑기위해서는
result[0] # 이렇게 뽑아야함

'eskim'

In [76]:
cur = conn.cursor()
cur.execute(sql,keyword+'%')
result = cur.fetchall() # tuple의 tuple로 반환
result # tuple안의 tuple 이므로 원하는 하나의 데이터를 뽑기위해서는
result[0][0] # 이렇게 뽑아야함

'eskim'

In [77]:
cur.close()
conn.close()